##### 0) 라이브러리 불러오기

In [1]:
# (1) 기본 라이브러리
import numpy as np

# (2) 머신러닝 라이브러리
import torch
import torch.optim as optim

In [2]:
torch.manual_seed(0)

##### 1) 데이터 불러오기

In [3]:
sentence = " if you want you"

In [4]:
sentence

' if you want you'

##### 2) Unique 단어장 생성

In [5]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [6]:
char_set

['i', 'n', 'w', 't', 'y', 'a', 'f', ' ', 'u', 'o']

In [7]:
char_dic

{'i': 0,
 'n': 1,
 'w': 2,
 't': 3,
 'y': 4,
 'a': 5,
 'f': 6,
 ' ': 7,
 'u': 8,
 'o': 9}

##### 3) 하이퍼파라미터 정의

In [8]:
input_size = len(char_dic) # dic_size = input_size
hidden_size = len(char_dic)
learning_rate = 0.1

In [9]:
input_size

10

In [10]:
hidden_size

10

##### 4) 데이터 전처리

(1) String -> Int

In [11]:
sentence_idx = [char_dic[c] for c in sentence]

In [12]:
sentence_idx

[7, 0, 6, 7, 4, 9, 8, 7, 2, 5, 1, 3, 7, 4, 9, 8]

(2) Data Split

In [13]:
x_data = [sentence_idx[:-1]]
x_one_hot = [np.eye(input_size)[x] for x in x_data] # np.eye : 대각행렬 생성 !!!
# x_data : [[7, 0, 6, 7, 4, 9, 8, 7, 2, 5, 1, 3, 7, 4, 9]]
# x : [7, 0, 6, 7, 4, 9, 8, 7, 2, 5, 1, 3, 7, 4, 9]
y_data = [sentence_idx[1:]]

In [15]:
x_data

[[7, 0, 6, 7, 4, 9, 8, 7, 2, 5, 1, 3, 7, 4, 9]]

In [14]:
x_data[0]

[7, 0, 6, 7, 4, 9, 8, 7, 2, 5, 1, 3, 7, 4, 9]

In [16]:
x_one_hot

[array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])]

In [17]:
y_data

[[0, 6, 2, 5, 1, 8, 2, 4, 7, 9, 3, 2, 5, 1, 8]]

(3) Numpy -> Tensor

In [18]:
x_train = torch.FloatTensor(x_one_hot) # X = x_train
y_train = torch.LongTensor(y_data) # Y = y_train

C:\Users\Bang\AppData\Local\Temp\ipykernel_11476\1449697340.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  x_train = torch.FloatTensor(x_one_hot) # X = x_train


In [19]:
x_train

tensor([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [20]:
y_train

tensor([[0, 6, 2, 5, 1, 8, 2, 4, 7, 9, 3, 2, 5, 1, 8]])

##### 5) 모델 선언 (RNN)

In [21]:
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)

##### 6) Optimizer + Loss Function 정의

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

##### 7) 모델 학습

In [23]:
# (1) Epoch 순회
for i in range(50):
    # 1] 모델 예측
    y_pred, _status = rnn(x_train) # outputs = y_pred !!
    """
    print(f"y_pred.size() : {y_pred.size()}") # torch.Size([1, 15, 10])
    print(f"input_size : {input_size}") # 10
    print(f"y_pred.view(-1, input_size).size() : {y_pred.view(-1, input_size).size()}") # torch.Size([15, 10])
    print(f"y_pred.view(-1, input_size)[0] : {y_pred.view(-1, input_size)[0]}") # tensor([-0.0171, -0.0729, -0.3786, -0.1471, -0.3150,  0.0425, -0.1634,  0.1294, 0.4553, -0.4327], grad_fn=<SelectBackward0>)
    print(f"y_train.view(-1).size : {y_train.view(-1).size()}") # torch.Size([15])
    """
    # 2] Cost 계산
    loss = criterion(y_pred.view(-1, input_size), y_train.view(-1)) # torch.Size([15, 10]) torch.Size([15]) !!
    # 3] 모델 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 4] 예측값 Int
    pred_data_int = y_pred.data.numpy().argmax(axis=2) # !!!
    # print(f"y_pred.data.numpy() : {y_pred.data.numpy()}")
    """
    print(f"y_pred.data.numpy().shape : {y_pred.data.numpy().shape}") # (1, 15, 10)
    [[[-1.71125513e-02 -7.29188621e-02 -3.78617078e-01 -1.47121057e-01
   -3.15001905e-01  4.24881615e-02 -1.63370296e-01  1.29351497e-01
    4.55255002e-01 -4.32749778e-01]
  [ 1.24736831e-01 -1.13964137e-02  9.05403048e-02 -4.39402670e-01
   -2.14903831e-01  3.63781452e-01 -1.29730999e-01  4.87463214e-02
   -2.46172413e-01 -1.42561672e-02]
  [ 2.33795792e-01 -1.52646691e-01 -1.97055906e-01 -5.96513629e-01
   -2.50488669e-01 -2.04520389e-01 -2.03721538e-01  4.34204340e-01
    3.62169623e-01 -3.44355851e-01]
  [ 9.08672363e-02 -2.46648952e-01 -3.08087349e-01 -2.29552597e-01
   -2.43646070e-01 -1.82804212e-01  4.59638387e-02  2.41613969e-01
    1.82456031e-01 -2.84000635e-01]
  [-1.96086377e-01 -1.48719087e-01  1.75950885e-01 -4.55239326e-01
   -4.17368203e-01  3.01247239e-01 -1.17449053e-01  3.98365319e-01
   -1.87145740e-01  7.93532748e-03]
  [ 2.37461314e-01  1.90436840e-04 -5.21515369e-01 -1.68140098e-01
    4.89513464e-02 -2.44426936e-01 -5.28508760e-02 -1.32374480e-01
    2.26483047e-01 -1.36129409e-01]
  [ 2.65214711e-01  1.02960326e-01 -1.28169462e-01 -3.25212777e-02
...
  [ 2.13628367e-01 -5.89405000e-02 -5.67303181e-01 -1.97684824e-01
    1.23873204e-01 -3.20607126e-01 -5.15337214e-02 -1.28327519e-01
    3.06786418e-01 -1.25176266e-01]]]
    print(f"pred_data_int : {pred_data_int}") # [[8 5 7 7 7 0 0 8 5 7 7 8 8 7 8]] !!
    print(f"pred_data_int.shape : {pred_data_int.shape}") # (1, 15)
    print(f"np.squeeze(pred_data_int).shape : {np.squeeze(pred_data_int).shape}") # (15,)
    """
    # 5] 예측값 Str
    pred_data_str = ''.join([char_set[c] for c in np.squeeze(pred_data_int)]) # !!!
    
    # 6] 결과 출력
    print("epoch : ", i, "loss : ", loss.item(), "pred_data_int : ", pred_data_int, "true_data : ", y_data, "pred_data_str : ", pred_data_str)

epoch :  0 loss :  2.2513201236724854 pred_data_int :  [[8 5 7 8 7 0 8 5 7 0 0 7 8 7 0]] true_data :  [[0, 6, 2, 5, 1, 8, 2, 4, 7, 9, 3, 2, 5, 1, 8]] pred_data_str :  uyauaiuyaiiauai
epoch :  1 loss :  1.9099316596984863 pred_data_int :  [[8 6 2 5 1 8 7 4 1 4 3 2 8 1 8]] true_data :  [[0, 6, 2, 5, 1, 8, 2, 4, 7, 9, 3, 2, 5, 1, 8]] pred_data_str :  uf youawowt uou
epoch :  2 loss :  1.6451270580291748 pred_data_int :  [[5 6 2 5 1 8 2 5 5 4 3 2 5 1 8]] true_data :  [[0, 6, 2, 5, 1, 8, 2, 4, 7, 9, 3, 2, 5, 1, 8]] pred_data_str :  yf you yywt you
epoch :  3 loss :  1.474958896636963 pred_data_int :  [[5 6 2 5 1 8 2 4 1 9 3 2 5 1 8]] true_data :  [[0, 6, 2, 5, 1, 8, 2, 4, 7, 9, 3, 2, 5, 1, 8]] pred_data_str :  yf you wont you
epoch :  4 loss :  1.3666033744812012 pred_data_int :  [[5 6 2 5 1 8 2 4 1 9 3 2 5 1 8]] true_data :  [[0, 6, 2, 5, 1, 8, 2, 4, 7, 9, 3, 2, 5, 1, 8]] pred_data_str :  yf you wont you
epoch :  5 loss :  1.2658944129943848 pred_data_int :  [[5 6 2 5 1 8 2 4 1 9 3 2 5 1 8